In [52]:
import os
import pandas as pd
import datetime as dt
import logging
import numpy as np
from numpy import random
import gensim
import nltk
nltk.download('all')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/pranay/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package

In [91]:
flares = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]


rindi = pd.read_csv('rindia.csv')
rindi = rindi.drop(['created'], axis=1)
rindi.head()

,title,url,comms_num,flare,score,id,author,body,timestamp
0,Coronavirus (COVID-19) Megathread - News and U...,https://www.reddit.com/r/india/comments/fqqdsg...,8884,AskIndia,344,fqqdsg,IAmMohit,###[Covid-19 Fundraisers & Donation Links](htt...,2020-03-29 08:40:33
1,Coronavirus (Covid-19) Multi-Lingual Resources...,https://www.reddit.com/r/india/comments/fo0xj9...,49,AskIndia,214,fo0xj9,IAmMohit,This post will serve as a wiki for multi-lingu...,2020-03-24 20:53:48
2,The wealth inequality in India is truly horrif...,https://i.redd.it/3rol63nk35s41.jpg,671,AskIndia,8288,fyyx8c,sinsandtonic,NaN,2020-04-11 20:47:54
3,It's sad what passes for breaking news these days,https://i.imgur.com/F0UHNcw.jpg,37,AskIndia,519,fz6vdl,godzillaweener,NaN,2020-04-12 04:25:31
4,Railways transports 20 litres of camel milk to...,https://twitter.com/PTI_News/status/1248965677...,47,AskIndia,666,fz59yf,aviakki1,NaN,2020-04-12 03:00:18


In [65]:

def get_date(created):
    return dt.datetime.fromtimestamp(created)

def string_form(value):
    return str(value)

def clean_text(text):
   
    text = BeautifulSoup(text, "lxml").text
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text


In [54]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))



In [69]:
rindi['title'] = rindi['title'].apply(string_form)
rindi['body'] = rindi['body'].apply(string_form)
rindi['comms_num'] = rindi['comms_num'].apply(string_form)

rindi['title'] = rindi['title'].apply(clean_text)
rindi['body'] = rindi['body'].apply(clean_text)
rindi['comms_num'] = rindi['comms_num'].apply(clean_text)

feature_combine = rindi["title"] + rindi["comms_num"] + rindi["url"]
rindi = rindi.assign(feature_combine = feature_combine)

In [79]:

def nb_classifier(X_train, X_test, y_train, y_test):
  
  from sklearn.naive_bayes import MultinomialNB


  nb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB()),
                ])
  nb.fit(X_train, y_train)

  y_pred = nb.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flares))

In [78]:
def train_test(X,y):
 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

  print("Results of Naive Bayes Classifier")
  nb_classifier(X_train, X_test, y_train, y_test)